In [1]:
import pandas as pd
import numpy as np

In [2]:
from IPython.core.magic import register_line_magic

@register_line_magic
def C(line):
    from IPython.core.getipython import get_ipython
    from fnmatch import fnmatch

    line = line.strip()
    idx_space = line.index(' ')
    space_num = line[:idx_space]
    if space_num.isdecimal():
        space_num = int(space_num)
        line = line[idx_space:]
    else:
        space_num = 5

    output_dict = {}
    cmds = line.split(';')
    for cmd in cmds:
        cmd = cmd.strip()
        output_dict[cmd] = repr(eval(cmd)).split("\n")

    str_maxlen_in_cols = [max(len(cmd), len(max(data, key=len))) for cmd, data in output_dict.items()]
    data_row_max = max([len(v) for v in output_dict.values()])

    out_lines = [""]*(data_row_max+2)

    space=''
    for i, (cmd, data) in enumerate(output_dict.items()):
        w = str_maxlen_in_cols[i]

        out_lines[0]+=space+f'{cmd:^{w}}'
        out_lines[1]+=space+"-"*w
        for j, d in enumerate(data, 2):
            out_lines[j]+=space+f'{d:{w}}'

        if len(data) < data_row_max:
            for j in range(len(data)+2, data_row_max+2):
                out_lines[j]+=space+' '*w
        
        space = ' '*space_num

    for line in out_lines:
        print(line)

## 時間序列

Pandas 提供了表示 `時間點`、`時間段` 和 `時間間隔` 等三種與時間有關的型態，以及元素為這些型態的索引物件，並提供了許多時間序列相關的函數。本節簡介一些與時間相關的物件和函數。在本章最後一節還會介紹一些相關的實例。

### 時間點、時間段、時間間隔

`Timestamp` 物件從 python 標準函數庫中的 `datetime` 類別繼承，表示時間軸上的一個時刻。它提供了方便的時區轉換功能。

下面呼叫 `Timestamp.now()` 取得目前時間 now，它是不包含時區資訊的本機時間。

呼叫其 `tz_localize()` 可以獲得指定時區的 `Timestamp` 物件。

而帶時區資訊的 `Timestamp` 物件可以透過其 `tz_convert()` 轉換時區。

下面的 now_shanghai 的時間以 "+08:00" 的結尾，表示它是東八區的時間，將其轉為東京時間獲得 now_tokyo ，它是東九區的時間：

In [3]:
now = pd.Timestamp.now()
now_shanghai = now.tz_localize("Asia/Shanghai")
now_tokyo = now_shanghai.tz_convert("Asia/Tokyo")
print( u"本機時間:", now )
print("-"*20)
print( u"上海時區:", now_shanghai )
print("-"*20)
print( u"東京時區:", now_tokyo )

本機時間: 2022-01-20 08:48:21.033496
--------------------
上海時區: 2022-01-20 08:48:21.033496+08:00
--------------------
東京時區: 2022-01-20 09:48:21.033496+09:00


不同時區的時間可以比較，而本機時間和時區時間無法比較：

In [4]:
now_shanghai == now_tokyo

True

透過 `pytz` 模組的 `common_timezones()` 可以獲得常用的表示時區的字串:

In [5]:
import pytz
# %omit pytz.common_timezones
pytz.common_timezones

['Africa/Abidjan', 'Africa/Accra', 'Africa/Addis_Ababa', 'Africa/Algiers', 'Africa/Asmara', 'Africa/Bamako', 'Africa/Bangui', 'Africa/Banjul', 'Africa/Bissau', 'Africa/Blantyre', 'Africa/Brazzaville', 'Africa/Bujumbura', 'Africa/Cairo', 'Africa/Casablanca', 'Africa/Ceuta', 'Africa/Conakry', 'Africa/Dakar', 'Africa/Dar_es_Salaam', 'Africa/Djibouti', 'Africa/Douala', 'Africa/El_Aaiun', 'Africa/Freetown', 'Africa/Gaborone', 'Africa/Harare', 'Africa/Johannesburg', 'Africa/Juba', 'Africa/Kampala', 'Africa/Khartoum', 'Africa/Kigali', 'Africa/Kinshasa', 'Africa/Lagos', 'Africa/Libreville', 'Africa/Lome', 'Africa/Luanda', 'Africa/Lubumbashi', 'Africa/Lusaka', 'Africa/Malabo', 'Africa/Maputo', 'Africa/Maseru', 'Africa/Mbabane', 'Africa/Mogadishu', 'Africa/Monrovia', 'Africa/Nairobi', 'Africa/Ndjamena', 'Africa/Niamey', 'Africa/Nouakchott', 'Africa/Ouagadougou', 'Africa/Porto-Novo', 'Africa/Sao_Tome', 'Africa/Tripoli', 'Africa/Tunis', 'Africa/Windhoek', 'America/Adak', 'America/Anchorage', 'Amer

`Period` 物件表示一個標準的時間段，例如某年、某月、某日、某小時等。時間段的長短由 `freq` 屬性決定。下面的程式呼叫 `Period.now()` ，分別獲得包含目前時間的 日週期時間段 和 小時週期時間段。

In [6]:
now_day = pd.Period.now(freq="D")
now_hour = pd.Period.now(freq="H")
%C now_day; now_hour
# print(now_day)
# print(now_hour)

         now_day                         now_hour            
-------------------------     -------------------------------
Period('2022-01-20', 'D')     Period('2022-01-20 08:00', 'H')


`freq` 屬性是一個描述時間段的字串，其可選值可以透過下面的程式獲得：

In [7]:
from pandas.tseries import frequencies
frequencies._period_code_map.keys()
frequencies._period_alias_dictionary();

AttributeError: module 'pandas.tseries.frequencies' has no attribute '_period_code_map'

對於週期為 年度 和 星期 的時間段，可以透過 `freq` 指定開始的時間。例如 `"W"` 表示以 `星期天` 開始的星期時間段，而 `"W-MON"` 則表示以 `星期一` 開始的星期時間段：

In [8]:
now_week_sun = pd.Period.now(freq="W")
now_week_mon = pd.Period.now(freq="W-MON")
%C now_week_sun; now_week_mon
# print(now_week_sun)
# print(now_week_mon)

              now_week_sun                                 now_week_mon              
----------------------------------------     ----------------------------------------
Period('2022-01-17/2022-01-23', 'W-SUN')     Period('2022-01-18/2022-01-24', 'W-MON')


時間段的起點和終點可以透過 `start_day` 和 `end_day` 屬性獲得，它們都是表示時間點的 `Timestamp` 物件：

In [9]:
%C now_day.start_time; now_day.end_time
# print(now_day.start_time)
# print(now_day.end_time)

       now_day.start_time                         now_day.end_time             
--------------------------------     ------------------------------------------
Timestamp('2022-01-20 00:00:00')     Timestamp('2022-01-20 23:59:59.999999999')


呼叫 `Timestamp` 物件的 `to_preiod()` 方法可以把時間點轉為包含該時間點的時間段。注意時間段不包含時區資訊：

In [10]:
now_shanghai.to_period("H")

<ipython-input-10-2f08a622c0d1>:1: UserWarning: Converting to Period representation will drop timezone information.
  now_shanghai.to_period("H")


Period('2022-01-20 08:00', 'H')

`Timestamp` 和 `Period` 物件可以透過其屬性獲得 年、月、日 等資訊。下面分別獲得 年、月、日、星期幾、一年中的第幾天、小時 等資訊：

In [11]:
%C now.year; now.month; now.day; now.dayofweek; now.dayofyear; now.hour
# print(now.year)
# print(now.month)
# print(now.day)
# print(now.dayofweek)
# print(now.dayofyear)
# print(now.hour)

now.year     now.month     now.day     now.dayofweek     now.dayofyear     now.hour
--------     ---------     -------     -------------     -------------     --------
2022         1             20          3                 20                8       


將兩個時間點相減可以獲得表示時間間隔的 `Timedelta` 物件，下面計算目前時刻離 2015 年國慶日還有多少時間：

In [12]:
national_day = pd.Timestamp("2015-10-1")
td = national_day - pd.Timestamp.now()
td

Timedelta('-2304 days +15:10:34.412446')

時間點 和 時間間隔 之間可以進行加減運算：

In [13]:
national_day + pd.Timedelta("20 days 10:20:30") 

Timestamp('2015-10-21 10:20:30')

`Timedelta` 物件的 `days`, `seconds`, `microseconds`, `nanoseconds` 等屬性分別獲得它包含的 天數、秒數、微秒數 和 毫微秒數。

注意這些值與對應的單位相乘並求和才是該物件表示的總時間間隔：

In [14]:
%C td.days; td.seconds; td.microseconds
# print(td.days)
# print(td.seconds)
# print(td.microseconds)

td.days     td.seconds     td.microseconds
-------     ----------     ---------------
-2304       54634          412446         


也可以透過關鍵字參數直接指定時間間隔的天數、小時數、分鐘數和秒數：

In [15]:
print( pd.Timedelta(days=10, hours=1, minutes=2, seconds=10.5) )
print( pd.Timedelta(seconds=100000) )

10 days 01:02:10.500000
1 days 03:46:40


### 時間序列

上節介紹的 `Timestamp`, `Period`, `Timedelta` 物件都是表示單一值的物件，這些值可以放在索引或資料列中。下面的程式呼叫 `random_timestamps()` 建之一個包含 5 個隨機時間點的 `DatetimeIndex` 物件 ts_index，然後透過 ts_index 建立 `PeriodIndex` 型態的索引物件 pd_index 和 `TimedeltaIndex` 型態的索引物件 td_index。`DatetimeIndex`, `PeriodIndex`, `TimedeltaIndex` 都從 `Index` 繼承，可以作為 `Series` 或 `DataFrame` 的索引。

`random_timestamps()` 中的 `date_range()` 函數建立以 `start` 為起點、以 `end` 為終點、週期為 `freq` 的 `DatetimeIndex` 物件。

In [16]:
def random_timestamps(start, end, freq, count):
    index = pd.date_range(start, end, freq=freq)
    locations = np.random.choice(np.arange(len(index)), size=count, replace=False)
    locations.sort()
    return index[locations]

np.random.seed(42)
ts_index = random_timestamps("2015-01-01", "2015-10-01", freq="Min", count=5)
pd_index = ts_index.to_period("M")
td_index = pd.TimedeltaIndex(np.diff(ts_index))

print( ts_index, "\n" )
print( pd_index, "\n" )
print( td_index, "\n" )

DatetimeIndex(['2015-01-15 16:12:00', '2015-02-15 08:04:00',
               '2015-02-28 12:30:00', '2015-08-06 02:40:00',
               '2015-08-18 13:13:00'],
              dtype='datetime64[ns]', freq=None) 

PeriodIndex(['2015-01', '2015-02', '2015-02', '2015-08', '2015-08'], dtype='period[M]', freq='M') 

TimedeltaIndex(['30 days 15:52:00', '13 days 04:26:00', '158 days 14:10:00',
                '12 days 10:33:00'],
               dtype='timedelta64[ns]', freq=None) 



下面檢視這三種索引物件的 `dtype` 屬性。其中 `M8[ns]` 和 `m8[ns]` 是 numpy 中表示時間點和時間間隔的 `dtype` 型態，內部採用 64 位元整數儲存時間資訊，其中 `[ns]` 表示時間的最小單位為毫微秒，能表示的時間範圍大約是西元1678年到西元2262年。`PeriodIndex` 也使用 64 位元整數，但是最小時間單位由其 `freq` 屬性決定。

In [17]:
%C ts_index.dtype; pd_index.dtype; td_index.dtype

 ts_index.dtype      pd_index.dtype      td_index.dtype 
----------------     --------------     ----------------
dtype('<M8[ns]')     period[M]          dtype('<m8[ns]')


這三種索引物件都提供了許多與時間相關的屬性，例如：

In [18]:
%C ts_index.weekday; pd_index.month; td_index.seconds

             ts_index.weekday                                pd_index.month                                      td_index.seconds                    
------------------------------------------     ------------------------------------------     -------------------------------------------------------
Int64Index([3, 6, 5, 3, 1], dtype='int64')     Int64Index([1, 2, 2, 8, 8], dtype='int64')     Int64Index([57120, 15960, 51000, 37980], dtype='int64')


`DataetimeIndex.shift(n, freq)` 可以移動時間點，將目前的時間移動 `n` 個 `freq` 時間單位。對於天數、小時這樣的精確單位，結果相當於與指定的時間間隔相加：

In [21]:
print(ts_index)
ts_index.shift(1, "H")

DatetimeIndex(['2015-01-15 16:12:00', '2015-02-15 08:04:00',
               '2015-02-28 12:30:00', '2015-08-06 02:40:00',
               '2015-08-18 13:13:00'],
              dtype='datetime64[ns]', freq=None)


DatetimeIndex(['2015-01-15 17:12:00', '2015-02-15 09:04:00',
               '2015-02-28 13:30:00', '2015-08-06 03:40:00',
               '2015-08-18 14:13:00'],
              dtype='datetime64[ns]', freq=None)

而對於月份這樣不精確的時間單位，則移動一個單位相當於移到月頭或月底：

In [22]:
print(ts_index)
ts_index.shift(1, "M")

DatetimeIndex(['2015-01-15 16:12:00', '2015-02-15 08:04:00',
               '2015-02-28 12:30:00', '2015-08-06 02:40:00',
               '2015-08-18 13:13:00'],
              dtype='datetime64[ns]', freq=None)


DatetimeIndex(['2015-01-31 16:12:00', '2015-02-28 08:04:00',
               '2015-03-31 12:30:00', '2015-08-31 02:40:00',
               '2015-08-31 13:13:00'],
              dtype='datetime64[ns]', freq=None)

`DatetimeIndex.normalize()` 將時刻修改為當天的凌晨零點，可以視為按日期取整數：

In [23]:
ts_index.normalize()

DatetimeIndex(['2015-01-15', '2015-02-15', '2015-02-28', '2015-08-06',
               '2015-08-18'],
              dtype='datetime64[ns]', freq=None)

如果希望對任意的時間週期取整數，可以先透過 `to_period()` 將其轉為 `PeriodIndex` 物件，然後再呼叫 `to_timestamp()` 方法轉換回 `DatetimeIndex` 物件。

`to_timestamp()` 的 `how` 參數決定將時間段的起點還是終點轉為時間點，預設值為 `"start"`。

In [24]:
ts_index.to_period("H").to_timestamp()

DatetimeIndex(['2015-01-15 16:00:00', '2015-02-15 08:00:00',
               '2015-02-28 12:00:00', '2015-08-06 02:00:00',
               '2015-08-18 13:00:00'],
              dtype='datetime64[ns]', freq=None)

下面的 `Series` 物件 `ts_series` 的索引為 `DatetimeIndex` 物件，這種 `Series` 物件被稱為時間序列：

In [26]:
ts_series = pd.Series(range(5), index=ts_index)
ts_series

2015-01-15 16:12:00    0
2015-02-15 08:04:00    1
2015-02-28 12:30:00    2
2015-08-06 02:40:00    3
2015-08-18 13:13:00    4
dtype: int64

時間序列提供一些專門用於處理時間的方法，例如 `between_time()` 傳回所有位於指定時間範圍之內的資料:

In [27]:
ts_series.between_time("9:00", "18:00")

2015-01-15 16:12:00    0
2015-02-28 12:30:00    2
2015-08-18 13:13:00    4
dtype: int64

`tshift()` 將索引移動指定的時間：

PS： 未來將不在支援 `tshift()` ，請改用 `shift()`

In [28]:
ts_series.tshift(1, freq="D")

<ipython-input-28-a5446b25ab38>:1: FutureWarning: tshift is deprecated and will be removed in a future version. Please use shift instead.
  ts_series.tshift(1, freq="D")


2015-01-16 16:12:00    0
2015-02-16 08:04:00    1
2015-03-01 12:30:00    2
2015-08-07 02:40:00    3
2015-08-19 13:13:00    4
dtype: int64

以 `PeriodIndex` 和 `TimedeltaIndex` 為索引的序列也可以使用 `tshift()` 對索引進行移動：

In [29]:
pd_series = pd.Series(range(5), index=pd_index)
td_series = pd.Series(range(4), index=td_index)
%C pd_series.tshift(1); td_series.tshift(10, freq="H")

 pd_series.tshift(1)      td_series.tshift(10, freq="H")
---------------------     ------------------------------
2015-02    0              31 days 01:52:00     0        
2015-03    1              13 days 14:26:00     1        
2015-03    2              159 days 00:10:00    2        
2015-09    3              12 days 20:33:00     3        
2015-09    4              dtype: int64                  
Freq: M, dtype: int64                                   


<string>:1: FutureWarning: tshift is deprecated and will be removed in a future version. Please use shift instead.


時間資訊除了可以作為索引之外，還可以作為 `Series` 或 `DataFrame` 的列。下面分別將上述三種索引物件轉為 `Series` 物件，並檢視其 `dtype` 屬性：

In [30]:
ts_data = pd.Series(ts_index)
pd_data = pd.Series(pd_index)
td_data = pd.Series(td_index)
%C ts_data.dtype; pd_data.dtype; td_data.dtype

 ts_data.dtype       pd_data.dtype      td_data.dtype  
----------------     -------------     ----------------
dtype('<M8[ns]')     period[M]         dtype('<m8[ns]')


可以看到 Pandas 的 `Series` 物件目前尚不支援使用 64 位元整數表示時間段，因此使用物件陣列儲存所有的 `Period` 物件。而對於時間點和時間間隔資料則採用 64 位元整數陣列儲存。

當序列的值為時間資料時，可以透過命名空間物件 `dt` 呼叫時間相關的屬性和方法。例如：

In [31]:
%C ts_data.dt.hour; pd_data.dt.month; td_data.dt.days

ts_data.dt.hour     pd_data.dt.month     td_data.dt.days
---------------     ----------------     ---------------
0    16             0    1               0     30       
1     8             1    2               1     13       
2    12             2    2               2    158       
3     2             3    8               3     12       
4    13             4    8               dtype: int64   
dtype: int64        dtype: int64                        
